In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('YOUR DATA PATH !!!!!!!')
df = df[['question', 'context', 'answer']]
df.head()

,question,context,answer
0,POST //SqlIn.asp HTTP/1.1 Connection: KeepAli...,If any string appears between select and from ...,SQL Injection
1,POST //md5.asp HTTP/1.1 Connection: KeepAlive...,If any string appears between select and from ...,SQL Injection
2,POST //plus/index.asp HTTP/1.1 Connection: Ke...,If any string appears between select and from ...,SQL Injection
3,POST //AspCms_Config.asp HTTP/1.1 Connection:...,If any string appears between select and from ...,SQL Injection
4,POST //plus/config.asp HTTP/1.1 Connection: K...,If any string appears between select and from ...,SQL Injection


In [4]:
# answer_start, answer_end 계산
df['answer_start'] = df.apply(lambda row: row['context'].find(row['answer']), axis=1)
df['answer_end'] = df.apply(lambda row: row['answer_start'] + len(row['answer']), axis=1)

In [5]:
df.head()

,question,context,answer,answer_start,answer_end
0,POST //SqlIn.asp HTTP/1.1 Connection: KeepAli...,If any string appears between select and from ...,SQL Injection,78,91
1,POST //md5.asp HTTP/1.1 Connection: KeepAlive...,If any string appears between select and from ...,SQL Injection,78,91
2,POST //plus/index.asp HTTP/1.1 Connection: Ke...,If any string appears between select and from ...,SQL Injection,78,91
3,POST //AspCms_Config.asp HTTP/1.1 Connection:...,If any string appears between select and from ...,SQL Injection,78,91
4,POST //plus/config.asp HTTP/1.1 Connection: K...,If any string appears between select and from ...,SQL Injection,78,91


In [6]:
import json

# JSON으로 변환
json_data = json.dumps({
    'data': [
        {
            'paragraphs': [
                {
                    'context': row['context'],
                    'qas': [
                        {
                            'question': row['question'],
                            'id': str(idx),
                            'answers': [
                                {
                                    'text': row['answer'],
                                    'answer_start': row['answer_start'],
                                    'answer_end': row['answer_end']
                                }
                            ]
                        }
                    ]
                }
            ]
        } for idx, row in df.iterrows()
    ]
})

In [8]:
# json_data

In [9]:
file_path = 'json_data의 file path'
file_name = 'json_data의 file name'

In [10]:
import os
# JSON 파일 저장
# with open(os.path.join(file_path, file_name), 'w', encoding='utf-8') as f:
#    f.write(json_data)